In [10]:
'''Implementation of PCY'''
import pandas as pd
import itertools
import time

#pass 1 - iterate through basket array, count quantity for each item and store in dict, create hashmap
def pass1(basketArray):
    itemCount = {}
    bucketCount = {}
    for i in range(len(basketArray)):
        lines = str(basketArray[i]).strip(' \n').split(' ') #string of items in basket
        hashPairs = list(itertools.combinations(lines, 2)) 
        
        for j in lines:
            if j in itemCount:
                itemCount[j] += 1
            else:
                itemCount[j] = 1

        #hash pair to bucket
        for k in range(len(hashPairs)):
            hashedVal = hash(hashPairs[k])
               
            if hashedVal in bucketCount:
                bucketCount[hashedVal] += 1
            else:
                bucketCount[hashedVal] = 1
    return itemCount, bucketCount
    

#returns bitmap of buckets
def bitmap(bucketCount, support):
    for l in bucketCount:
        if bucketCount[l] > support:
            bucketCount[l] = 1
        else:
            bucketCount[l] = 0
    return bucketCount
    

#check if each item in list is frequent
def checkFrequent(countList, support):
    frequentList = []
    for i in countList:
        if countList[i] >= support:
            frequentList.append(i)
    return set(frequentList)
    

#pass 2 - find pairs where both elements are frequent and bucket values = 1
def pass2(basketArray, frequentItems):
    candidateCount = {}

    for a in range(len(basketArray)):
        lines = str(basketArray[a]).strip(' \n').split(' ') #string of items in basket
        freqItems = [i for i in lines if i in frequentItems]
        pairsList = list(itertools.combinations (freqItems, 2)) #iterable list of pairs

        #compares pairs made from frequent items and pairs made from whole basket list
        for i in pairsList:
            if i in candidateCount:
                candidateCount[i] += 1
            else:
                candidateCount[i] = 1        
    return candidateCount
    

def PCY(fileName, supp): 
    #read in file
    baskets = pd.read_csv(fileName, sep = '\t', header = None) 
    basketArray = baskets.values.ravel()
    support = int(supp * len(baskets)) 

    #pass 1
    itemCount, bucketCount = pass1(basketArray)
 
    #intermediate step
    frequentItems = checkFrequent(itemCount, support)
    bucketCount = bitmap(bucketCount, support)
            
    #if no items are above support threshold
    if len(frequentItems) == 0 or len(bucketCount) == 0:
        print("No frequent items")
        return

    #pass 2
    candidateCount = pass2(basketArray, frequentItems)
    
    #getting frequent pairs
    frequentPairs = list(checkFrequent(candidateCount, support)) 

    for b in range(len(frequentPairs)):
        tempHash = hash(frequentPairs[b])
     
        if bucketCount[tempHash] == 0:
            frequentPairs.remove(b)
        else: 
            pass
    
    #writing list to txt file
    with open('frequentPairsPCY.txt','w+') as f:
        f.write(" ".join(map(str, frequentPairs)))
    return frequentPairs

st = time.time()   
frequentPairs = PCY("retail.txt", 0.02)
print("frequent pairs:", len(frequentPairs))
et = time.time()
execution = et - st
print(execution)

22
7.743791103363037
